# LSTM Classification with SUBJ Dataset
<hr>

We will build a text classification model using LSTM model on the SUBJ Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/SUBJ/SUBJ.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10000, 3)


,sentence,label,split
0,"smart and alert , thirteen conversations about...",0,train
1,"color , musical bounce and warm seas lapping o...",0,train
2,it is not a mass market entertainment but an u...,0,train
3,a light hearted french film about the spiritua...,0,train
4,my wife is an actress has its moments in looki...,0,train
...,...,...,...
9995,"in the end , they discover that balance in lif...",1,train
9996,a counterfeit 1000 tomin bank note is passed i...,1,train
9997,enter the beautiful and mysterious secret agen...,1,train
9998,after listening to a missionary from china spe...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10000 non-null  object
 1   label     10000 non-null  int32 
 2   split     10000 non-null  object
dtypes: int32(1), object(2)
memory usage: 195.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5000,5000
1,5000,5000


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [9]:
sentences[0]

'smart and alert , thirteen conversations about one thing is a small gem .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [10]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more .
Into a sequence of int: [336, 208, 8, 16, 921, 25, 29, 312, 7, 313, 32, 2, 488, 551, 5, 3203, 7, 2, 129, 194, 10, 8, 60, 2330, 716, 39, 10, 128, 43, 82, 54, 81, 45]
Into a padded sequence: [ 336  208    8   16  921   25   29  312    7  313   32    2  488  551
    5 3203    7    2  129  194   10    8   60 2330  716   39   10  128
   43   82   54   81   45    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [12]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
in 7
is 8
's 9
it 10
21324


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## LSTM Model

In [13]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [14]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               140544    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


In [15]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [16]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/15
282/282 [==============================] - 58s 149ms/step - loss: 0.4524 - accuracy: 0.7798 - val_loss: 0.2054 - val_accuracy: 0.9190
Epoch 2/15
282/282 [==============================] - 42s 150ms/step - loss: 0.0677 - accuracy: 0.9792 - val_loss: 0.2482 - val_accuracy: 0.9120
Epoch 3/15
282/282 [==============================] - 39s 139ms/step - loss: 0.0128 - accuracy: 0.9961 - val_loss: 0.3429 - val_accuracy: 0.9070
Epoch 4/15
282/282 [==============================] - 36s 128ms/step - loss: 0.0069 - accuracy: 0.9985 - val_loss: 0.4184 - val_accuracy: 0.9110
Epoch 5/15
282/282 [==============================] - 36s 126ms/step - loss: 0.0018 - accuracy: 0.9990 - val_loss: 0.5232 - val_accuracy: 0.8990
Epoch 6/15
282/282 [==============================] - 36s 126ms/step - loss: 8.9157e-04 - accuracy: 0.9996 - val_loss: 0.3468 - val_accuracy: 0.9130
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 91.90000295639038
Training 

Test Accuracy: 92.59999990463257
Training 9: 
Epoch 1/15
282/282 [==============================] - 54s 149ms/step - loss: 0.4631 - accuracy: 0.7548 - val_loss: 0.2117 - val_accuracy: 0.9110
Epoch 2/15
282/282 [==============================] - 41s 146ms/step - loss: 0.0613 - accuracy: 0.9777 - val_loss: 0.2590 - val_accuracy: 0.8990
Epoch 3/15
282/282 [==============================] - 40s 142ms/step - loss: 0.0122 - accuracy: 0.9965 - val_loss: 0.4775 - val_accuracy: 0.8960
Epoch 4/15
282/282 [==============================] - 40s 143ms/step - loss: 0.0047 - accuracy: 0.9982 - val_loss: 0.4438 - val_accuracy: 0.9020
Epoch 5/15
282/282 [==============================] - 48s 170ms/step - loss: 0.0019 - accuracy: 0.9992 - val_loss: 0.4691 - val_accuracy: 0.8970
Epoch 6/15
282/282 [==============================] - 44s 156ms/step - loss: 6.6907e-04 - accuracy: 1.0000 - val_loss: 0.4628 - val_accuracy: 0.8960
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopp

## Summary

In [17]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,91.900003,92.1,92.500001,92.400002,91.600001,93.400002,92.1,92.6,91.100001,92.400002,92.210001


In [18]:
report = record
report = report.to_excel('GRU_SUBJ.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [19]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [21]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [22]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 17913 words present from 21324 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [23]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [24]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.02488821e-01, -1.44414252e-03,  1.11408719e+00, ...,
         2.96026624e-03, -1.23360697e+00,  1.51209989e+00],
       [ 1.13769531e-01,  1.79687500e-01, -2.65625000e-01, ...,
        -2.18750000e-01, -3.93066406e-02,  2.09960938e-01],
       [ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02, ...,
         1.08886719e-01, -1.01928711e-02,  2.07519531e-02],
       [ 1.08886719e-01, -1.66992188e-01,  8.98437500e-02, ...,
        -1.96289062e-01, -2.31445312e-01,  4.61425781e-02]])

## LSTM Model

In [25]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [26]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 140,673
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [27]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [28]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/40
282/282 [==============================] - 36s 75ms/step - loss: 0.4745 - accuracy: 0.7680 - val_loss: 0.2771 - val_accuracy: 0.8950
Epoch 2/40
282/282 [==============================] - 17s 61ms/step - loss: 0.2295 - accuracy: 0.9044 - val_loss: 0.2480 - val_accuracy: 0.9010
Epoch 3/40
282/282 [==============================] - 17s 61ms/step - loss: 0.1900 - accuracy: 0.9262 - val_loss: 0.2364 - val_accuracy: 0.9050
Epoch 4/40
282/282 [==============================] - 17s 61ms/step - loss: 0.1687 - accuracy: 0.9339 - val_loss: 0.2632 - val_accuracy: 0.8950
Epoch 5/40
282/282 [==============================] - 19s 67ms/step - loss: 0.1307 - accuracy: 0.9493 - val_loss: 0.2636 - val_accuracy: 0.9000
Epoch 6/40
282/282 [==============================] - 20s 70ms/step - loss: 0.1194 - accuracy: 0.9579 - val_loss: 0.2619 - val_accuracy: 0.9010
Epoch 7/40
282/282 [==============================] - 18s 63ms/step - loss: 0.0957 - accuracy: 0.9647 - val_loss: 0.2959 - val_accuracy:

282/282 [==============================] - 17s 60ms/step - loss: 0.0488 - accuracy: 0.9837 - val_loss: 0.9939 - val_accuracy: 0.7960
Epoch 11/40
282/282 [==============================] - 17s 60ms/step - loss: 0.0353 - accuracy: 0.9884 - val_loss: 0.9962 - val_accuracy: 0.7920
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 88.80000114440918
Training 5: 
Epoch 1/40
282/282 [==============================] - 33s 75ms/step - loss: 0.4818 - accuracy: 0.7594 - val_loss: 0.2907 - val_accuracy: 0.8750
Epoch 2/40
282/282 [==============================] - 17s 60ms/step - loss: 0.2240 - accuracy: 0.9109 - val_loss: 0.2751 - val_accuracy: 0.8810
Epoch 3/40
282/282 [==============================] - 17s 61ms/step - loss: 0.1831 - accuracy: 0.9276 - val_loss: 0.2425 - val_accuracy: 0.9010
Epoch 4/40
282/282 [==============================] - 18s 62ms/step - loss: 0.1476 - accuracy: 0.9448 - val_loss: 0.2394 - val_accuracy: 0.9050
Epoch 5/40
282/2

Test Accuracy: 86.50000095367432
Training 8: 
Epoch 1/40
282/282 [==============================] - 33s 76ms/step - loss: 0.4895 - accuracy: 0.7376 - val_loss: 0.4501 - val_accuracy: 0.8150
Epoch 2/40
282/282 [==============================] - 17s 61ms/step - loss: 0.2097 - accuracy: 0.9169 - val_loss: 0.6113 - val_accuracy: 0.7790
Epoch 3/40
282/282 [==============================] - 17s 61ms/step - loss: 0.1884 - accuracy: 0.9277 - val_loss: 0.3958 - val_accuracy: 0.8370
Epoch 4/40
282/282 [==============================] - 17s 61ms/step - loss: 0.1581 - accuracy: 0.9429 - val_loss: 0.4993 - val_accuracy: 0.8080
Epoch 5/40
282/282 [==============================] - 17s 61ms/step - loss: 0.1410 - accuracy: 0.9472 - val_loss: 0.6257 - val_accuracy: 0.7930
Epoch 6/40
282/282 [==============================] - 17s 61ms/step - loss: 0.1189 - accuracy: 0.9564 - val_loss: 0.7218 - val_accuracy: 0.7820
Epoch 7/40
282/282 [==============================] - 18s 63ms/step - loss: 0.1015 - accur

## Summary

In [29]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,90.600002,89.600003,92.000002,88.800001,90.499997,91.900003,86.500001,83.700001,88.499999,88.599998,89.070001


In [30]:
report = record2
report = report.to_excel('GRU_SUBJ_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## LSTM Model

In [31]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [32]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [33]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [34]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/40
282/282 [==============================] - 56s 150ms/step - loss: 0.4327 - accuracy: 0.7908 - val_loss: 0.2142 - val_accuracy: 0.9160
Epoch 2/40
282/282 [==============================] - 41s 145ms/step - loss: 0.0996 - accuracy: 0.9672 - val_loss: 0.2975 - val_accuracy: 0.8970
Epoch 3/40
282/282 [==============================] - 41s 146ms/step - loss: 0.0247 - accuracy: 0.9935 - val_loss: 0.3186 - val_accuracy: 0.9030
Epoch 4/40
282/282 [==============================] - 41s 145ms/step - loss: 0.0048 - accuracy: 0.9990 - val_loss: 0.3898 - val_accuracy: 0.9100
Epoch 5/40
282/282 [==============================] - 43s 154ms/step - loss: 0.0027 - accuracy: 0.9998 - val_loss: 0.4400 - val_accuracy: 0.8980
Epoch 6/40
282/282 [==============================] - 52s 184ms/step - loss: 0.0034 - accuracy: 0.9987 - val_loss: 0.4816 - val_accuracy: 0.9030
Epoch 7/40
282/282 [==============================] - 42s 151ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.5274 - val_ac

282/282 [==============================] - 54s 190ms/step - loss: 0.0103 - accuracy: 0.9969 - val_loss: 0.3488 - val_accuracy: 0.9170
Epoch 5/40
282/282 [==============================] - 57s 202ms/step - loss: 0.0034 - accuracy: 0.9993 - val_loss: 0.4042 - val_accuracy: 0.9120
Epoch 6/40
282/282 [==============================] - 41s 145ms/step - loss: 8.5163e-04 - accuracy: 1.0000 - val_loss: 0.4383 - val_accuracy: 0.9120
Epoch 7/40
282/282 [==============================] - 44s 157ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.4709 - val_accuracy: 0.9030
Epoch 8/40
282/282 [==============================] - 39s 138ms/step - loss: 0.0016 - accuracy: 0.9997 - val_loss: 0.4019 - val_accuracy: 0.9070
Epoch 9/40
282/282 [==============================] - 41s 144ms/step - loss: 7.1222e-04 - accuracy: 1.0000 - val_loss: 0.4996 - val_accuracy: 0.9120
Epoch 10/40
282/282 [==============================] - 38s 133ms/step - loss: 9.0017e-05 - accuracy: 1.0000 - val_loss: 0.5345 - val_

282/282 [==============================] - 41s 145ms/step - loss: 3.8442e-04 - accuracy: 1.0000 - val_loss: 0.4452 - val_accuracy: 0.9030
Epoch 8/40
282/282 [==============================] - 41s 145ms/step - loss: 1.1858e-04 - accuracy: 1.0000 - val_loss: 0.4799 - val_accuracy: 0.9060
Epoch 9/40
282/282 [==============================] - 40s 142ms/step - loss: 7.3364e-05 - accuracy: 1.0000 - val_loss: 0.5178 - val_accuracy: 0.9060
Epoch 10/40
282/282 [==============================] - 41s 147ms/step - loss: 7.6635e-05 - accuracy: 1.0000 - val_loss: 0.5109 - val_accuracy: 0.9060
Epoch 11/40
282/282 [==============================] - 39s 137ms/step - loss: 3.7062e-05 - accuracy: 1.0000 - val_loss: 0.5343 - val_accuracy: 0.9030
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 92.69999861717224
Training 10: 
Epoch 1/40
282/282 [==============================] - 57s 154ms/step - loss: 0.4569 - accuracy: 0.7727 - val_loss: 0.2449 - val_accur

## Summary

In [35]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,91.600001,92.6,92.6,91.799998,92.500001,88.4,92.500001,88.999999,92.699999,90.200001,91.39


In [36]:
report = record3
report = report.to_excel('GRU_SUBJ_3.xlsx', sheet_name='dynamic')